In [1]:
import torch
import jax.numpy as jnp
import numpy as np
import jax

In [2]:
rng = np.random.default_rng(0)
operand = rng.normal(size=(64, 64)).astype(np.float32)
axis = 0

In [5]:
j_x = jnp.array(operand, dtype=jnp.bfloat16)
t_x = torch.from_numpy(operand).to(torch.bfloat16)

In [6]:
j_out = jax.lax.cumlogsumexp(j_x, axis=axis)
j_out

Array([[0.125977, -0.131836, 0.640625, ..., -1.17188, 1.74219, -0.496094],
       [0.925781, 0.5, 1.91406, ..., -0.257812, 1.78125, 0.640625],
       [1.125, 0.695312, 1.96094, ..., 0.332031, 1.80469, 0.773438],
       ...,
       [4.4375, 4.53125, 4.53125, ..., 4.15625, 4.40625, 4.46875],
       [4.46875, 4.59375, 4.53125, ..., 4.1875, 4.40625, 4.46875],
       [4.46875, 4.59375, 4.53125, ..., 4.21875, 4.40625, 4.46875]],      dtype=bfloat16)

In [7]:
t_out = torch.ops.aten.logcumsumexp.default(t_x, dim=axis)
t_out

tensor([[ 0.1260, -0.1318,  0.6406,  ..., -1.1719,  1.7422, -0.4961],
        [ 0.9258,  0.5000,  1.9141,  ..., -0.2578,  1.7812,  0.6445],
        [ 1.1250,  0.6953,  1.9609,  ...,  0.3320,  1.8047,  0.7773],
        ...,
        [ 4.4375,  4.5312,  4.5312,  ...,  4.1562,  4.4062,  4.4375],
        [ 4.4688,  4.5938,  4.5312,  ...,  4.1875,  4.4062,  4.4688],
        [ 4.4688,  4.5938,  4.5312,  ...,  4.2188,  4.4062,  4.4688]],
       dtype=torch.bfloat16)

In [8]:
j_out_np = np.array(j_out, dtype=np.float32)
t_out_np = t_out.detach().to("cpu").to(torch.float32).numpy()

In [9]:
j_out_np

array([[ 0.12597656, -0.13183594,  0.640625  , ..., -1.171875  ,
         1.7421875 , -0.49609375],
       [ 0.92578125,  0.5       ,  1.9140625 , ..., -0.2578125 ,
         1.78125   ,  0.640625  ],
       [ 1.125     ,  0.6953125 ,  1.9609375 , ...,  0.33203125,
         1.8046875 ,  0.7734375 ],
       ...,
       [ 4.4375    ,  4.53125   ,  4.53125   , ...,  4.15625   ,
         4.40625   ,  4.46875   ],
       [ 4.46875   ,  4.59375   ,  4.53125   , ...,  4.1875    ,
         4.40625   ,  4.46875   ],
       [ 4.46875   ,  4.59375   ,  4.53125   , ...,  4.21875   ,
         4.40625   ,  4.46875   ]], shape=(64, 64), dtype=float32)

In [10]:
t_out_np

array([[ 0.12597656, -0.13183594,  0.640625  , ..., -1.171875  ,
         1.7421875 , -0.49609375],
       [ 0.92578125,  0.5       ,  1.9140625 , ..., -0.2578125 ,
         1.78125   ,  0.64453125],
       [ 1.125     ,  0.6953125 ,  1.9609375 , ...,  0.33203125,
         1.8046875 ,  0.77734375],
       ...,
       [ 4.4375    ,  4.53125   ,  4.53125   , ...,  4.15625   ,
         4.40625   ,  4.4375    ],
       [ 4.46875   ,  4.59375   ,  4.53125   , ...,  4.1875    ,
         4.40625   ,  4.46875   ],
       [ 4.46875   ,  4.59375   ,  4.53125   , ...,  4.21875   ,
         4.40625   ,  4.46875   ]], shape=(64, 64), dtype=float32)

In [ ]:
diff = np.abs(j_out_np - t_out_np)
max_diff = np.nanmax(diff) if not np.isnan(diff).all() else 0.0

max_diff  # if gpu np.float32(0.15625)

np.float32(0.03125)

In [12]:
diff

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00390625],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00390625],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.03125   ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], shape=(64, 64), dtype=float32)